prepare dataset

In [ ]:
from train import prepare_dataset

INPUT_DIR = ''
ALIGNED_DIR = 'test_dataset/aligned'
MASKED_DIR = 'test_dataset/masked'
COMBINE_DIR = 'test_dataset/combine'

print('INFO: PREPARE DATASET')

print('INFO: Align face')
print('Please wait...')
prepare_dataset.aligned_dataset(INPUT_DIR, ALIGNED_DIR)
print('Aligned face save at ' + ALIGNED_DIR)
print('INFO: Remove face mask region')
print('Please wait...')
prepare_dataset.masked_dataset(ALIGNED_DIR, MASKED_DIR)
print('Masked face save at ' + MASKED_DIR)
print('INFO: Combine dataset')
print('Please wait...')
prepare_dataset.combine_dataset(ALIGNED_DIR, MASKED_DIR, COMBINE_DIR)
print('Combine dataset save at ' + COMBINE_DIR)

evaluate

In [ ]:
from train.evaluate import calculate_accuracy
from utils.featureExtraction import feature_extraction
import os
import numpy as np
from utils.classification import predict_cosine


DATASET_DIR = ALIGNED_DIR
THRESHOLDS = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
split_ratio = 0.2

print('Runnning forward pass on {DATASET_DIR} images')

embs = []
image_paths = []
labels = []
for fld_name in os.listdir(DATASET_DIR):
    image_path = os.path.join(DATASET_DIR, fld_name)
    image_paths.append(image_path)
    labels.append(fld_name)
    embs.append(feature_extraction(image_path))

num_class = len(labels)
image_indices = np.arange(num_class)
np.random.shuffle(image_indices)
split = int(round(num_class*(1-split_ratio)))

train_embs = [embs[i] for i in image_indices[0:split]]
train_labels = [labels[i] for i in image_indices[0:split]]
test_embs = [embs[i] for i in image_indices[split:-1]]
test_labels = [labels[i] for i in image_indices[0:split]]

max_indexes = []
max_probs = []
actual_issame_list = []

for i, audit_emb in enumerate(test_embs):
    max_index, max_prob = predict_cosine(audit_emb, train_embs)
    actual_issame = True if train_labels[max_index] == test_labels[i] else False
    max_indexes.append(max_index)
    max_probs.append(max_prob)
    actual_issame_list.append(actual_issame)

for threshold in THRESHOLDS:
    print('........................')
    print('Evaluate threshold: {threshold}')
    acc, val_rate, error_rate = calculate_accuracy(threshold, max_probs, actual_issame_list)
    print('Accuracy: {acc}')
    print('Validation rate: {val_rate}')
    print('Error Rate: {error_rate}')

                

    